In [ ]:
import os
import pandas as pd
%pip install ibm_db

# Show all columns
pd.set_option('display.max_columns', None)

# Show all rows
#pd.set_option('display.max_rows', None)

# (Optional) Adjust column width
pd.set_option('display.max_colwidth', None)


# ✅ Database connection string
conn_str = (
    "DATABASE=testdb1;"
    "HOSTNAME=<hostname>"
    "PORT=50000;"
    "PROTOCOL=TCPIP;"
    "UID=db2inst1;"
    "PWD=<password>;"
)

import ibm_db
import pandas as pd


# === Step 1: Define Input/Output Directories ===
sql_dir = '/Volumes/workspace/default/sql/sf'
output_dir = '/Volumes/workspace/default/output/sf'

# === Step 2: Connect to Db2 ===
conn = ibm_db.connect(conn_str, "", "")

# === Step 3: Process Each .sql File ===
for filename in os.listdir(sql_dir):
    if filename.lower().endswith('.sql'):
        sql_path = os.path.join(sql_dir, filename)

        # Read SQL text
        with open(sql_path, 'r') as f:
            sql_text = f.read()
        sql_text = sql_text.replace('<YOUR_SCHEMA>', UID.upper())
        print(f"🚀 Executing: {filename}")

        try:

            result = ibm_db.exec_immediate(conn, sql_text)

            rows = []
            while (row := ibm_db.fetch_assoc(result)):
                rows.append(row)

            df = pd.DataFrame(rows)
            if 'COLUMN_NAMES' in df.columns:
                df['COLUMN_NAMES'] = df['COLUMN_NAMES'].apply(lambda x: x.replace('+', '|') if isinstance(x, str) else x)

            # Build output filename
            base_name = os.path.splitext(filename)[0]
            output_filename = f"{base_name}_result.csv"
            output_path = os.path.join(output_dir, output_filename)

            # Save to CSV
            df.to_csv(output_path, index=False)
            print(f"✅ Saved: {output_filename}")

        except Exception as e:
            print(f"❌ Error executing {filename}: {e}")
# === Optional: Close the connection ===
ibm_db.close(conn)


